# Preprocessing data | recovery-news-data.csv

*CS 539 - Social Media Mining | Francesca Spezzano*

*Computer Science | Boise State University*

*11.23.2022 | Fall 2022*

*Aida Gomezbueno Berezo | aidagomezbuenobe@u.boisestate.edu*


Launching notebook with the following command:

> *jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e100000000000*

#### Importing libraries | Loading data

In [1]:
import datetime
import math
import nltk
import numpy as np
import num2words
import pandas as pd
import sklearn
import time
from datetime import datetime
from nltk.stem import *
from nltk.corpus import names, stopwords, words
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy import sparse

import warnings
warnings.filterwarnings("ignore")

In [2]:
dat = pd.read_csv(r'recovery-news-data.csv')
df = pd.DataFrame(dat)
df.columns = ['index', 'news_id', 'url', 'publisher', 'publish_date', 'author', 'title', 'image', 'body_text', 'political_bias', 'country', 'reliability']
df = df.drop('index', axis=1)
df.head()

,news_id,url,publisher,publish_date,author,title,image,body_text,political_bias,country,reliability
0,0,https://www.nytimes.com/article/what-is-corona...,The New York Times,2020-01-21,"['Knvul Sheikh', 'Roni Caryn Rabin']",The Coronavirus: What Scientists Have Learned ...,https://static01.nyt.com/images/2020/03/12/sci...,\nA novel respiratory virus that originated in...,Left,USA,1
1,1,https://www.npr.org/2020/01/22/798392172/chine...,National Public Radio (NPR),2020-01-22,['Emily Feng'],Chinese Health Officials: More Die From Newly ...,https://media.npr.org/include/images/facebook-...,Chinese Health Officials: More Die From Newly ...,Center,USA,1
2,2,https://www.theverge.com/2020/1/23/21078457/co...,The Verge,2020-01-23,['Nicole Wetsman'],Everything you need to know about the coronavirus,https://cdn.vox-cdn.com/thumbor/a9_Oz7cvSBKyal...,Public health experts around the globe are scr...,Left-center,USA,1
3,3,https://www.worldhealth.net/news/novel-coronav...,WorldHealth.Net,2020-01-24,[],Novel Coronavirus Cases Confirmed To Be Spreading,https://www.worldhealth.net/media/original_ima...,The first two coronavirus cases in Europe have...,NaN,USA,0
4,4,https://www.theverge.com/2020/1/24/21080845/co...,The Verge,2020-01-24,"['Nicole Wetsman', 'Zoe Schiffer', 'Jay Peters...",Coronavirus disrupts the world: updates on the...,https://cdn.vox-cdn.com/thumbor/t2gt1SmEni4Mcr...,"A new coronavirus appeared in Wuhan, China, at...",Left-center,USA,1


#### Cleaning data

In [3]:
clean_subs = ['https://', 'http://', 'www.']
substring = "/"
urls=[]
for i in df['url']:
    for j in clean_subs:
        if j in i:
            i = i.split(j)
            i = i[1]
    if substring in i:
        i = i.split(substring)
        i = i[0]
    urls.append(i)
df['url']=urls

df['publish_date'] = df['publish_date'].fillna("1970-11-01")
epoch = datetime.utcfromtimestamp(0)
for i in df['publish_date']:
    date_object = datetime.strptime(str(i), '%Y-%m-%d').date()
    delta = date_object - epoch.date()
    dt = datetime.fromtimestamp(int(delta.total_seconds())).strftime('%Y-%m-%d')
    mth = datetime.fromtimestamp(int(delta.total_seconds())).strftime('%B')
    if mth=="October":
        mth = "UNKNOWN"
    df['publish_date'] = df['publish_date'].replace(i, mth)
df['author'] = df['author'].fillna("")
replace_simb = ['[', ']', "'", ' etc.', "‘"]
coma = ", "
array_col = []
for i in df['author']:
    temp = []
    for j in replace_simb:
        if j in i:
            i = i.replace(j, "")
            
df['political_bias'] = df['political_bias'].fillna("NEUTRAL")
df['image'] = df['image'].fillna("")
df['title'] = df['title'].fillna("")
df['country'] = df['country'].fillna("UNKNOWN")

df['alltext'] = df['publisher'] + " " + df['author'] + " " +df['political_bias'] + " " + df['title'] + " " + df['body_text'] 

#### Sanity check

In [4]:
df.head()

,news_id,url,publisher,publish_date,author,title,image,body_text,political_bias,country,reliability,alltext
0,0,nytimes.com,The New York Times,January,"['Knvul Sheikh', 'Roni Caryn Rabin']",The Coronavirus: What Scientists Have Learned ...,https://static01.nyt.com/images/2020/03/12/sci...,\nA novel respiratory virus that originated in...,Left,USA,1,"The New York Times ['Knvul Sheikh', 'Roni Cary..."
1,1,npr.org,National Public Radio (NPR),January,['Emily Feng'],Chinese Health Officials: More Die From Newly ...,https://media.npr.org/include/images/facebook-...,Chinese Health Officials: More Die From Newly ...,Center,USA,1,National Public Radio (NPR) ['Emily Feng'] Cen...
2,2,theverge.com,The Verge,January,['Nicole Wetsman'],Everything you need to know about the coronavirus,https://cdn.vox-cdn.com/thumbor/a9_Oz7cvSBKyal...,Public health experts around the globe are scr...,Left-center,USA,1,The Verge ['Nicole Wetsman'] Left-center Every...
3,3,worldhealth.net,WorldHealth.Net,January,[],Novel Coronavirus Cases Confirmed To Be Spreading,https://www.worldhealth.net/media/original_ima...,The first two coronavirus cases in Europe have...,NEUTRAL,USA,0,WorldHealth.Net [] NEUTRAL Novel Coronavirus C...
4,4,theverge.com,The Verge,January,"['Nicole Wetsman', 'Zoe Schiffer', 'Jay Peters...",Coronavirus disrupts the world: updates on the...,https://cdn.vox-cdn.com/thumbor/t2gt1SmEni4Mcr...,"A new coronavirus appeared in Wuhan, China, at...",Left-center,USA,1,"The Verge ['Nicole Wetsman', 'Zoe Schiffer', '..."


#### Preprocessing | Basic Steps

    1. Lowercase. 
    2. Stop words.
    3. Punctuation/Symbols.
    4. Apostophre.
    5. Single characters.
    6. Stemming.
    7. Lemmatisation.
    8. Converting Numbers.

In [5]:
def toLowerCase(df):
    df['alltext'] = df['alltext'].str.lower()
    return df

def delStopWords(df):
    stop_words = nltk.corpus.stopwords.words('english')
    valid_allt=[]
    for i in df['alltext']:
        valid_t=[]
        for j in i:
            if j not in stop_words:
                valid_t.append(j)
        valid_allt.append(valid_t)
    df['alltext']=valid_allt
    return df

def toListOfLists(df):
    all_text = []
    for i in df['alltext']:
        i = i.split()
        all_text.append(i)
    df['alltext']=all_text
    return df  

def delSymbols(df):
    symbols = "!\"“‘’”#$%&()*+-./:;<,=>?@[\]^_`{|}~\n►●…"
    valid_allt=[]
    for i in df['alltext']:
        valid_t = []
        for j in i:
            for k in symbols:
                if k in j:
                    j = j.replace(k, "")
            valid_t.append(j)
        if "[]" in valid_t:
            valid_t.remove("[]")
        valid_allt.append(valid_t)
    df['alltext']=valid_allt
    return df

def delApostrophe(df):
    stop_words = nltk.corpus.stopwords.words('english')
    valid_alltext=[]
    for i in df['alltext']:
        valid_text=[]
        for j in i:
            if j not in stop_words:
                if "'" in j:
                    j=j.replace("'", "")
                valid_text.append(j)
        valid_alltext.append(valid_text)
    df['alltext']=valid_alltext
    return df

def delSingleChars(df):
    valid_allt=[]
    for i in df['alltext']:
        valid_t = []
        for j in i:
            if len(j)>1:
                valid_t.append(j)
        valid_allt.append(valid_t)
    df['alltext']=valid_allt
    return df

#NOMBRES PROPIOS CON PUNTOS.
def remainDomains(df):
    domain_dot=[]
    for i in df['publisher']:
        if "." in i:
            domain_dot.append(str(i).lower())
    domain_dot = np.unique(domain_dot)
    valid_allt=[]
    for i in df['alltext']:
        valid_t=[]
        for j in i:
            for k in domain_dot:
                if "." in k:
                    k_wodot = k.replace(".", "")
                    if k_wodot in j:
                        j = j.replace(j, k)
            valid_t.append(j)
        valid_allt.append(valid_t)
    df['alltext']=valid_allt    
    return df

#THE ORDER IS LEMMATIZATION THEN STEMMING, OR JUST STEMMING.
def lemmatize(df):
    wnl = WordNetLemmatizer()
    valid_allt=[]
    for i in df['alltext']:
        valid_t=[]
        for j in i:
            j = wnl.lemmatize(j)
            valid_t.append(j)
        valid_allt.append(valid_t) 
    df['alltext']=valid_allt
    return df

#STEMMING
def stem(df):
    stemmer = PorterStemmer()
    valid_allt=[]
    for i in df['alltext']:
        valid_t=[]
        for j in i:
            j = stemmer.stem(j)
            valid_t.append(j)
        valid_allt.append(valid_t) 
    df['alltext']=valid_allt
    return df
    #df.to_csv('preprocessing-numbers.csv', index=False)
    
#converting numbers
def num_conversion(j):
    k, l = j.split()
    k = float(k)
    l = float(l)
    j = str(k*l)
    return j

def convertNumbers(df):
    valid_allt=[]
    for i in df['alltext']:
        valid_t = []
        for j in i:
            if j.isnumeric():
                if "½" in j:
                    j = j.replace("½", " 0.5")
                    j = num_conversion(j)
                if "¼" in j:
                    j = j.replace("¼", " 0.25")
                    j = num_conversion(j)
                if "⅔" in j:
                    j = j.replace("⅔", " 0.67")
                    j = num_conversion(j)
                if "¾" in j:
                    j = j.replace("¾", " 0.75")
                    j = num_conversion(j)
                if "⅓" in j:
                    j = j.replace("⅓", " 0.33")
                    j = num_conversion(j)
                j = num2words.num2words(float(j))
            valid_t.append(j)          
        valid_allt.append(valid_t)
    df['alltext']=valid_allt
    return df
    #df.to_csv('preprocessing-no_numbers.csv', index=False)

def preprocess(df):
    df = toLowerCase(df)
    df = toListOfLists(df)
    df = delStopWords(df)
    df = delSymbols(df)
    df = delStopWords(df)
    df = delApostrophe(df)
    df = delSingleChars(df)
    df = lemmatize(df)
    df = stem(df)
    df = convertNumbers(df)
    df = delSymbols(df)
    df = remainDomains(df)

#### Sanity check

In [6]:
preprocess(df)
df.head()

,news_id,url,publisher,publish_date,author,title,image,body_text,political_bias,country,reliability,alltext
0,0,nytimes.com,The New York Times,January,"['Knvul Sheikh', 'Roni Caryn Rabin']",The Coronavirus: What Scientists Have Learned ...,https://static01.nyt.com/images/2020/03/12/sci...,\nA novel respiratory virus that originated in...,Left,USA,1,"[new, york, time, knvul, sheikh, roni, caryn, ..."
1,1,npr.org,National Public Radio (NPR),January,['Emily Feng'],Chinese Health Officials: More Die From Newly ...,https://media.npr.org/include/images/facebook-...,Chinese Health Officials: More Die From Newly ...,Center,USA,1,"[nation, public, radio, npr, emili, feng, cent..."
2,2,theverge.com,The Verge,January,['Nicole Wetsman'],Everything you need to know about the coronavirus,https://cdn.vox-cdn.com/thumbor/a9_Oz7cvSBKyal...,Public health experts around the globe are scr...,Left-center,USA,1,"[verg, nicol, wetsman, leftcent, everyth, need..."
3,3,worldhealth.net,WorldHealth.Net,January,[],Novel Coronavirus Cases Confirmed To Be Spreading,https://www.worldhealth.net/media/original_ima...,The first two coronavirus cases in Europe have...,NEUTRAL,USA,0,"[worldhealth.net, neutral, novel, coronaviru, ..."
4,4,theverge.com,The Verge,January,"['Nicole Wetsman', 'Zoe Schiffer', 'Jay Peters...",Coronavirus disrupts the world: updates on the...,https://cdn.vox-cdn.com/thumbor/t2gt1SmEni4Mcr...,"A new coronavirus appeared in Wuhan, China, at...",Left-center,USA,1,"[verg, nicol, wetsman, zoe, schiffer, jay, pet..."


#### TF-IDF Score
> #### TF (Term Frequency). TF is individual to each document and word. Frequency of a term in relation to the doc that belongs to.
> * TF(t,d) = count of t in d.

> #### DF (Document Frequency). Number of documents in which the word is present.
> * DF(t) = occurrence of t in N documents.

> #### IDF (Inverse Document Frequency). Informativeness of term t.
> * IDF(t) = log(D/df).

> #### TF-IDF (Term Frequency - Inverse Document Frequency).
> * TF-IDF(t,d) = tf(t,d)*log(N/df).

In [50]:
def getTF(df):
    tf_all = []
    n_docs = len(df['alltext'])
    n_words_per_doc = {}
    w = 0
    for i in df['alltext']:
        term_freq = {} 
        uniq_vals = pd.unique(i) 
        n_words = len(i) 
        n_words_per_doc[w] = n_words 
        w+=1
        n_uniq_vals = len(uniq_vals)
        for x in range(n_uniq_vals): 
            n=0
            temp = uniq_vals[x]
            for j in i:
                if j==temp: 
                    n+=1
            term_freq[temp]=n 
        tf_all.append(term_freq) 
    return df, n_docs, tf_all
    
def getDF(df, n_docs, tf_all):    
    words_ = []
    n=0
    for x in range(n_docs): 
        dict_ = tf_all[x] 
        for y in dict_.keys(): 
            words_.append(y)
            n+=1
    words = np.unique(words_) 
    nw = len(words) 
    n_news = len(tf_all)
    doc_freq={}
    for x in range(nw):
        n=0
        temp = words[x]
        for y in range(n_news):
            dict_ = tf_all[y]
            vals = dict_.keys()
            if temp in vals:
                n+=1 
                continue
        doc_freq[temp] = n
    return df, doc_freq, nw, words

def getIDF(df, doc_freq):
    idf={}
    n_docs = len(df['alltext'])
    for x in doc_freq.keys(): 
        df_=doc_freq.get(x)
        idf_=math.log(n_docs/(df_), 2)
        idf[x]=idf_
    return df, idf

def _getTFIDF(df, tf_all, idf):
    tf_idf_allt = []
    for x in range(len(tf_all)): 
        tf_idf={}
        dict_tf_doc = tf_all[x]
        for y in dict_tf_doc.keys():
            tf_r = dict_tf_doc.get(y)
            idf_r = idf.get(y)
            tf_idf_ = tf_r*idf_r
            tf_idf[y]=tf_idf_
        tf_idf_allt.append(tf_idf) 
    df['tf-idf']=tf_idf_allt
    return df, tf_idf_allt

def getTFIDF(nw, words, tf_idf_allt):
    dict_words = {}
    for x in range(nw):
        tfidf_scores = []
        temp = words[x]
        for y in tf_idf_allt:
            if temp in y:
                tfidf_scores.append(y.get(temp))
            else:
                tfidf_scores.append(0)
        dict_words[temp] = tfidf_scores
    df_tfidf = pd.DataFrame()
    for i in dict_words.keys():
        df_tfidf[i] = dict_words.get(i) #array
    return df, df_tfidf

def _TfidfVectorizer(df):
    df, n_docs, tf_all = getTF(df)
    df, doc_freq, nw, words = getDF(df, n_docs, tf_all)
    df, idf = getIDF(df, doc_freq)
    df, tf_idf_allt = _getTFIDF(df, tf_all, idf)
    df, df_tfidf = getTFIDF(nw, words, tf_idf_allt)
    return df_tfidf

def _TfidfVectorizer_(df):
    df, n_docs, tf_all = getTF(df)
    df, doc_freq, nw, words = getDF(df, n_docs, tf_all)
    df, idf = getIDF(df, doc_freq)
    df, tf_idf_allt = _getTFIDF(df, tf_all, idf)
    df, df_tfidf = getTFIDF(nw, words, tf_idf_allt)
    return df_tfidf

## TfidfVectorizer w/ Sparse Matrix

*In order to be able to train our model with the previous TF-IDF we've computed, we should first convert the DataFrame into a **sparse matrix** (as sklearn does with its TfidfVectorizer). After that, we'll be able to train, test, and finally getting an accuracy higher than 90%.*

In [51]:
df_tfidf_as_sparse_matrix = sparse.csr_matrix(_TfidfVectorizer(df))

X = df_tfidf_as_sparse_matrix.toarray()
y = df['reliability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = LogisticRegression(random_state = 0)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print("Accuracy: %.2f%%" % (accuracy_score(pred, y_test)*100))

Accuracy: 90.31%


In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X = []
for each in df['alltext']:
    X.append(' '.join(each))
X = vectorizer.fit_transform(X)
X = X.toarray()
y = df['reliability']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = LogisticRegression(random_state = 0)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print("Accuracy: %.2f%%" % (accuracy_score(pred, y_test)*100))

Accuracy: 82.43%
